In [1]:
# %% [code]
"""
================================================================================
   VESUVIUS V27 - TUNED HYSTERESIS ENSEMBLE

   - Base: V24 (Best Score: 0.537)
   - Optimization 1: Ensemble Weights adjusted to 0.75/0.25 (Favoring ComboLoss)
   - Optimization 2: Z_RADIUS increased to 2 (Better vertical connectivity)
   - Optimization 3: T_HIGH lowered to 0.80 (Improved Recall)
   - Constraint: Kept Overlap 0.25 & 4x TTA to ensure <9h runtime
================================================================================
"""

import os
import sys
import subprocess
from IPython.display import clear_output

# ============================================================================
# 1. SETUP PACKAGES
# ============================================================================
var = "/kaggle/input/vsdetection-packages-offline-installer-only/whls"
if os.path.exists(var):
    print(f"Installing packages from: {var}")
    subprocess.run(
        f"pip install {var}/keras_nightly-*.whl {var}/tifffile-*.whl {var}/imagecodecs-*.whl {var}/medicai-*.whl --no-index --find-links {var}",
        shell=True, check=True
    )
    clear_output()
else:
    print(f"❌ ERROR: Package directory not found at: {var}")
    sys.exit(1)

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Protobuf patch
try:
    from google.protobuf import message_factory as _message_factory
    if not hasattr(_message_factory.MessageFactory, "GetPrototype"):
        from google.protobuf.message_factory import GetMessageClass
        def _GetPrototype(self, descriptor):
            return GetMessageClass(descriptor)
        _message_factory.MessageFactory.GetPrototype = _GetPrototype
except:
    pass

import keras
from keras import ops
from medicai.transforms import Compose, NormalizeIntensity 
from medicai.models import TransUNet
from medicai.utils.inference import SlidingWindowInference

import numpy as np
import pandas as pd
import zipfile
import tifffile
import scipy.ndimage as ndi
from skimage.morphology import remove_small_objects
import gc

print("="*60)
print("VESUVIUS V27 - OPTIMIZED HYSTERESIS")
print("="*60)

# ============================================================================
# CONFIG
# ============================================================================
root_dir = "/kaggle/input/vesuvius-challenge-surface-detection"
test_dir = f"{root_dir}/test_images"
output_dir = "/kaggle/working/submission_masks"
zip_path = "/kaggle/working/submission.zip"
os.makedirs(output_dir, exist_ok=True)

# Model config
NUM_CLASSES = 3
PATCH_SIZE = (160, 160, 160)
OVERLAP = 0.25      # Sharp inference
BATCH_SIZE = 2      # Speed optimization

# Post-processing config (Tuned V24)
T_LOW = 0.45        # Keep connectivity permissive
T_HIGH = 0.80       # Lowered from 0.85 to find more seeds
Z_RADIUS = 2        # Increased from 1 to 2 (Fixes vertical splits)
XY_RADIUS = 0       # Keep 0 (Prevent mergers)
DUST_MIN_SIZE = 300 # Lowered from 500 to keep smaller valid fragments

# ============================================================================
# MODEL LOADING
# ============================================================================
def get_ensemble_models():
    base_path = "/kaggle/input/vsd-model/keras/transunet"
    
    # 1. The "Super Model" (V3 ComboLoss)
    path_combo = f"{base_path}/3/transunet.seresnext50.160px.comboloss.weights.h5"
    # 2. The "Support Model" (V2 Default) - Actually the TPU model path from V24
    # Let's use the path we used in V24 which worked:
    path_tpu = "/kaggle/input/train-vesuvius-surface-3d-detection-on-tpu/model.weights.h5"
    
    models = []
    
    if os.path.exists(path_combo):
        print(f"Loading Model 1 (ComboLoss): {path_combo}")
        m = TransUNet(input_shape=(160, 160, 160, 1), encoder_name='seresnext50', classifier_activation=None, num_classes=NUM_CLASSES)
        m.load_weights(path_combo)
        models.append(m)
        
    if os.path.exists(path_tpu):
        print(f"Loading Model 2 (TPU): {path_tpu}")
        m = TransUNet(input_shape=(160, 160, 160, 1), encoder_name='seresnext50', classifier_activation=None, num_classes=NUM_CLASSES)
        m.load_weights(path_tpu)
        models.append(m)
        
    if not models:
        print("WARNING: No weights found. Initializing random model.")
        m = TransUNet(input_shape=(160, 160, 160, 1), encoder_name='seresnext50', classifier_activation=None, num_classes=NUM_CLASSES)
        return [m]
        
    return models

# ============================================================================
# TRANSFORMS
# ============================================================================
def val_transformation(image):
    data = {"image": image}
    pipeline = Compose([
        NormalizeIntensity(keys=["image"], nonzero=True, channel_wise=False)
    ])
    result = pipeline(data)
    return result["image"]

def load_volume(path):
    vol = tifffile.imread(path).astype(np.float32)
    vol = vol[None, ..., None]
    return vol

# ============================================================================
# INFERENCE LOGIC (4x TTA)
# ============================================================================
def predict_with_tta(inputs, swi):
    logits = []
    # Original
    logits.append(swi(inputs))
    # Rotations
    for k in [1, 2, 3]:
        img_r = np.rot90(inputs, k=k, axes=(2, 3))
        p = swi(img_r)
        p = np.rot90(p, k=-k, axes=(2, 3))
        logits.append(p)
    return np.mean(logits, axis=0)

def ensemble_predict(inputs, models):
    # Weighted Ensemble (Aligned with 0.546 solution)
    if len(models) == 2:
        weights = [0.75, 0.25] # Stronger weight for ComboLoss
    else:
        weights = [1.0]
        
    ensemble_probs = []
    
    for i, model in enumerate(models):
        swi = SlidingWindowInference(
            model, 
            num_classes=NUM_CLASSES, 
            roi_size=PATCH_SIZE, 
            sw_batch_size=BATCH_SIZE, 
            mode='gaussian', 
            overlap=OVERLAP
        )
        
        logits = predict_with_tta(inputs, swi)
        probs = ops.softmax(logits, axis=-1)
        fg_probs = probs[..., 1] 
        ensemble_probs.append(fg_probs * weights[i])
    
    final_probs = np.sum(ensemble_probs, axis=0)
    return np.squeeze(final_probs)

# ============================================================================
# POST-PROCESSING (HYSTERESIS)
# ============================================================================
def build_anisotropic_struct(z_radius, xy_radius):
    z, r = z_radius, xy_radius
    if z == 0 and r == 0: return None
    depth = 2 * z + 1
    size = 2 * r + 1
    struct = np.zeros((depth, size, size), dtype=bool)
    cz, cy, cx = z, r, r
    for dz in range(-z, z + 1):
        for dy in range(-r, r + 1):
            for dx in range(-r, r + 1):
                if dy * dy + dx * dx <= r * r:
                    struct[cz + dz, cy + dy, cx + dx] = True
    return struct

def topo_postprocess(probs, T_low, T_high, z_radius, xy_radius, dust_min_size):
    # 1. Hysteresis
    strong = probs >= T_high
    weak = probs >= T_low
    
    if not strong.any(): return np.zeros_like(probs, dtype=np.uint8)
    
    struct_hyst = ndi.generate_binary_structure(3, 3)
    mask = ndi.binary_propagation(strong, mask=weak, structure=struct_hyst)
    
    if not mask.any(): return np.zeros_like(probs, dtype=np.uint8)

    # 2. Anisotropic Closing
    struct = build_anisotropic_struct(z_radius, xy_radius)
    if struct is not None:
        mask = ndi.binary_closing(mask, structure=struct)

    # 3. Dust Removal
    if dust_min_size > 0:
        mask = remove_small_objects(mask.astype(bool), min_size=dust_min_size)

    return mask.astype(np.uint8)

# ============================================================================
# MAIN PIPELINE
# ============================================================================
print("\nLoading models...")
models = get_ensemble_models()
test_df = pd.read_csv(f"{root_dir}/test.csv")

print("\nStarting Inference...")
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for idx, row in test_df.iterrows():
        image_id = row["id"]
        print(f"\n[{idx+1}/{len(test_df)}] Processing {image_id}...")
        
        vol = load_volume(f"{test_dir}/{image_id}.tif")
        vol = val_transformation(vol)
        
        probs = ensemble_predict(vol, models)
        
        final_mask = topo_postprocess(
            probs, 
            T_low=T_LOW, 
            T_high=T_HIGH, 
            z_radius=Z_RADIUS, 
            xy_radius=XY_RADIUS, 
            dust_min_size=DUST_MIN_SIZE
        )
        
        print(f"    Foreground voxels: {final_mask.sum():,}")
        
        out_path = f"{output_dir}/{image_id}.tif"
        tifffile.imwrite(out_path, final_mask)
        z.write(out_path, arcname=f"{image_id}.tif")
        os.remove(out_path)
        
        del vol, probs, final_mask
        gc.collect()

print("\n" + "="*60)
print("V27 COMPLETE")
print(f"Submission: {zip_path}")
print("="*60)

2026-02-09 09:43:32.207533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770630212.643584      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770630212.804433      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


VESUVIUS V27 - OPTIMIZED HYSTERESIS

Loading models...
Loading Model 1 (ComboLoss): /kaggle/input/vsd-model/keras/transunet/3/transunet.seresnext50.160px.comboloss.weights.h5


I0000 00:00:1770630235.217564      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770630235.220257      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



Starting Inference...

[1/1] Processing 1407735...


Total patch 27:   0%|          | 0/14 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1770630251.579407      70 service.cc:148] XLA service 0x7f305800c880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770630251.580994      70 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1770630251.581013      70 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1770630253.047464      70 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1770630256.713353      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1770630256.946017      70 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate

    Foreground voxels: 3,208,295

V27 COMPLETE
Submission: /kaggle/working/submission.zip
